In [ ]:
import os
import glob
import pandas as pd
import re # For more robust number extraction
import matplotlib.pyplot as plt
import matplotlib.style as style
from mpl_toolkits.mplot3d import Axes3D # For 3D plotting
import numpy as np # For potential numerical operations, e.g. np.nan
import matplotlib.dates as mdates
from datetime import datetime, timedelta

In [ ]:
def combine_log_files(data_folder_path: str) -> pd.DataFrame:
    """
    Reads all LOG_*.TXT files from the specified folder, sorts them numerically,
    and combines them into a single Pandas DataFrame.

    Args:
        data_folder_path (str): The path to the folder containing the .txt log files.

    Returns:
        pd.DataFrame: A DataFrame containing all combined data, or an empty
                      DataFrame if no suitable files are found or an error occurs.
    """
    if not os.path.isdir(data_folder_path):
        print(f"Error: Folder '{data_folder_path}' not found.")
        return pd.DataFrame()

    all_files_in_folder = glob.glob(os.path.join(data_folder_path, "*"))
    
    txt_files = []
    for f_path in all_files_in_folder:
        filename = os.path.basename(f_path)
        if filename.upper().startswith("LOG_") and filename.upper().endswith(".TXT"):
            txt_files.append(f_path)

    if not txt_files:
        print(f"No LOG_*.TXT files found in '{data_folder_path}'.")
        return pd.DataFrame()

    def sort_key(filepath):
        filename = os.path.basename(filepath)
        match = re.search(r'LOG_(\d+)\.TXT', filename, re.IGNORECASE)
        if match:
            return int(match.group(1))
        return float('inf') 

    sorted_files = sorted(txt_files, key=sort_key)
    
    print("Files to be processed in order:")
    for f in sorted_files:
        print(f"  - {os.path.basename(f)}")

    all_dataframes = []
    for file_path in sorted_files:
        try:
            df = pd.read_csv(file_path)
            all_dataframes.append(df)
            print(f"Successfully read and processed: {os.path.basename(file_path)}")
        except pd.errors.EmptyDataError:
            print(f"Warning: File {os.path.basename(file_path)} is empty and will be skipped.")
        except Exception as e:
            print(f"Error reading file {os.path.basename(file_path)}: {e}")

    if not all_dataframes:
        print("No data could be read from the files.")
        return pd.DataFrame()

    combined_df = pd.concat(all_dataframes, ignore_index=True)
    return combined_df


In [ ]:
input_folder = "Data_In" 
output_folder = "Data_Out" # Define the output folder


combined_data = combine_log_files(input_folder)
df = combined_data
if not combined_data.empty:
    print("\n--- Combined DataFrame ---")
    print("Shape:", combined_data.shape)
    print("\nHead:")
    print(combined_data.head())

In [ ]:
os.makedirs(output_folder, exist_ok=True)
print(f"\nSaving plots to folder: {output_folder}")

# Apply a publication-ready style
style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

# --- Data Preparation for Plot 1 Time Axis ---
df_plotting = df.copy()

gps_datetime_cols = ['GPS_Year', 'GPS_Month', 'GPS_Day', 'GPS_Hour', 'GPS_Min', 'GPS_Sec']
for col in gps_datetime_cols:
    df_plotting[col] = pd.to_numeric(df_plotting[col], errors='coerce')

df_plotting['GPS_DateTime_Valid'] = pd.NaT

valid_gps_mask = (
    (df_plotting['GPS_Fix'] == 1) &
    (df_plotting['GPS_Year'] > 0) &
    (df_plotting['GPS_Month'] >= 1) & (df_plotting['GPS_Month'] <= 12) &
    (df_plotting['GPS_Day'] >= 1) & (df_plotting['GPS_Day'] <= 31) &
    (df_plotting['GPS_Hour'] >= 0) & (df_plotting['GPS_Hour'] <= 23) &
    (df_plotting['GPS_Min'] >= 0) & (df_plotting['GPS_Min'] <= 59) &
    (df_plotting['GPS_Sec'] >= 0) & (df_plotting['GPS_Sec'] <= 59)
)

datetime_constructor_df = pd.DataFrame({
    'year': df_plotting.loc[valid_gps_mask, 'GPS_Year'],
    'month': df_plotting.loc[valid_gps_mask, 'GPS_Month'],
    'day': df_plotting.loc[valid_gps_mask, 'GPS_Day'],
    'hour': df_plotting.loc[valid_gps_mask, 'GPS_Hour'],
    'minute': df_plotting.loc[valid_gps_mask, 'GPS_Min'],
    'second': df_plotting.loc[valid_gps_mask, 'GPS_Sec']
})

if not datetime_constructor_df.empty:
    # Ensure all columns are numeric before passing to to_datetime
    for col in datetime_constructor_df.columns:
        datetime_constructor_df[col] = pd.to_numeric(datetime_constructor_df[col], errors='coerce')
    
    # Drop rows with any NaNs that might have been introduced by coerce, before to_datetime
    datetime_constructor_df.dropna(inplace=True)

    if not datetime_constructor_df.empty:
         # Assign back to the original DataFrame's filtered view (df_plotting.loc[valid_gps_mask])
         # The index of datetime_constructor_df matches the True values in valid_gps_mask
        df_plotting.loc[datetime_constructor_df.index, 'GPS_DateTime_Valid'] = pd.to_datetime(datetime_constructor_df, errors='coerce')


df_plotting['Plot_Time'] = pd.NaT

if not df_plotting['GPS_DateTime_Valid'].isnull().all():
    first_valid_dt_idx = df_plotting['GPS_DateTime_Valid'].first_valid_index()
    if first_valid_dt_idx is not None:
        df_plotting.loc[first_valid_dt_idx, 'Plot_Time'] = df_plotting.loc[first_valid_dt_idx, 'GPS_DateTime_Valid']
        # Backfill time for rows before the first valid GPS time
        for i in range(first_valid_dt_idx - 1, -1, -1):
            # Check if previous Plot_Time is already set (e.g. by another valid GPS point)
            # This check is mostly for robustness, standard loop should be fine
            if pd.isna(df_plotting.loc[i+1, 'Plot_Time']): # Should not happen in this loop
                break 
            df_plotting.loc[i, 'Plot_Time'] = df_plotting.loc[i + 1, 'Plot_Time'] - pd.Timedelta(seconds=1)
        
        # Fill subsequent NaNs or use GPS time
        for i in range(first_valid_dt_idx + 1, len(df_plotting)):
            if pd.notna(df_plotting.loc[i, 'GPS_DateTime_Valid']):
                df_plotting.loc[i, 'Plot_Time'] = df_plotting.loc[i, 'GPS_DateTime_Valid']
            else:
                if pd.isna(df_plotting.loc[i-1, 'Plot_Time']): # If previous is also NaN, cannot increment
                     # This could happen if there's a large gap of non-GPS data after first_valid_dt_idx
                     # and before another valid GPS_DateTime_Valid.
                     # For now, let it be NaT, to be caught by the manual input if all Plot_Time is NaT.
                     # Or, one could decide to keep incrementing.
                     # If this creates an issue, this logic might need refinement for sparse GPS data.
                     pass # Keep as NaT
                else:
                     df_plotting.loc[i, 'Plot_Time'] = df_plotting.loc[i-1, 'Plot_Time'] + pd.Timedelta(seconds=1)
    # else: # first_valid_dt_idx is None, but not all GPS_DateTime_Valid are null. This is unlikely.
        # If this happens, Plot_Time will remain all NaT, and the manual input logic will trigger.
    #    pass

# MODIFIED SECTION FOR MANUAL TIME INPUT
if df_plotting['Plot_Time'].isnull().all():
    print("Warning: No valid GPS time found in any row to anchor the timeline, or issues populating Plot_Time.")
    print("The time axis for plots will be based on a manually entered start time.")
    while True:
        manual_start_time_str = input("Please enter the start datetime of the experiment (e.g., 'YYYY-MM-DD HH:MM:SS'): ")
        try:
            start_time = pd.to_datetime(manual_start_time_str)
            print(f"Using manually entered start time: {start_time} for the time axis.")
            df_plotting['Plot_Time'] = [start_time + pd.Timedelta(seconds=i) for i in range(len(df_plotting))]
            break 
        except ValueError:
            print("Invalid datetime format. Please use a format like 'YYYY-MM-DD HH:MM:SS' (e.g., '2023-01-01 14:30:00'). Try again.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}. Please check your input and try again.")

df_plotting['BMP_PressAtm'] = df_plotting['BMP_PressPa'] / 101325.0

In [ ]:
# --- 1) Time Series Subplots ---
print("Generating Plot 1: Time Series Data...")
fig1, axes1 = plt.subplots(4, 1, figsize=(15, 12), sharex=True)
fig1.suptitle('Sensor Time Series Data', fontsize=16, y=0.99)

axes1[0].plot(df_plotting['Plot_Time'], df_plotting['SHT_TempC'], label='SHT Temp', color='crimson')
axes1[0].set_ylabel('Temperature (°C)')
axes1[0].legend(loc='upper right')
axes1[0].set_title('Ambient Temperature (SHT85)')

axes1[1].plot(df_plotting['Plot_Time'], df_plotting['SHT_Hum_%'], label='SHT Humidity', color='royalblue')
axes1[1].set_ylabel('Humidity (%)')
axes1[1].legend(loc='upper right')
axes1[1].set_title('Ambient Humidity (SHT85)')

axes1[2].plot(df_plotting['Plot_Time'], df_plotting['BMP_PressAtm'], label='BMP Pressure', color='green')
axes1[2].set_ylabel('Pressure (atm)')
axes1[2].legend(loc='upper right')
axes1[2].set_title('Atmospheric Pressure (BMP585)')

axes1[3].plot(df_plotting['Plot_Time'], df_plotting['CO2_ppm'], label='CO2', color='purple')
axes1[3].set_ylabel('CO2 (ppm)')
axes1[3].set_xlabel('Time')
axes1[3].legend(loc='upper right')
axes1[3].set_title('CO2 Concentration (SCD30)')

if pd.api.types.is_datetime64_any_dtype(df_plotting['Plot_Time']):
    fig1.autofmt_xdate() 
    axes1[3].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))

plt.tight_layout(rect=[0, 0, 1, 0.97])
plot1_path = os.path.join(output_folder, "plot1_time_series.png")
plt.savefig(plot1_path)
print(f"Saved: {plot1_path}")
plt.show() # MODIFIED: Show plot
# plt.close(fig1) # MODIFIED: Commented out to show plot in Jupyter



In [ ]:
%matplotlib inline

In [ ]:
# --- Data Preprocessing ---
print("Preprocessing data: calculating H2O ppm, removing outliers, and preparing for aggregation...")
df_processed = df_plotting.copy()

# Ensure Plot_Time is datetime and set as index
if not pd.api.types.is_datetime64_any_dtype(df_processed['Plot_Time']):
    df_processed['Plot_Time'] = pd.to_datetime(df_processed['Plot_Time'])
df_processed = df_processed.set_index('Plot_Time')

# --- *** ADDED: Calculate Water Vapor Concentration (ppm) *** ---
# Constants
ATM_TO_KPA = 101.325 # Conversion factor from atm to kPa

# Ensure required columns exist
required_cols_for_ppm = ['SHT_TempC', 'SHT_Hum_%', 'BMP_PressAtm']
if all(col in df_processed.columns for col in required_cols_for_ppm):
    print("Calculating Water Vapor Concentration (H2O ppm)...")
    # Extract data (use .values for slight speedup with numpy, though pandas ops are fine too)
    T_C = df_processed['SHT_TempC']
    RH_percent = df_processed['SHT_Hum_%']
    P_atm = df_processed['BMP_PressAtm']

    # 1. Calculate Saturation Vapor Pressure (es) in kPa using Magnus-Tetens approximation
    # es(T) = 0.6108 * exp((17.27 * T) / (T + 237.3))  (T in Celsius, es in kPa)
    es_kPa = 0.6108 * np.exp((17.27 * T_C) / (T_C + 237.3))

    # 2. Calculate Actual Vapor Pressure (e) in kPa
    # e = (RH / 100) * es
    e_kPa = (RH_percent / 100.0) * es_kPa

    # 3. Convert Total Atmospheric Pressure (P) to kPa
    P_total_kPa = P_atm * ATM_TO_KPA

    # 4. Calculate Water Vapor Concentration in ppm (volume/volume)
    # ppm_v = (e / P_total) * 1,000,000
    # Ensure P_total_kPa is not zero to avoid division errors (though unlikely for atm pressure)
    # Replace potential division by zero or NaN pressure with NaN result
    with np.errstate(divide='ignore', invalid='ignore'): # Suppress warnings for division by zero/NaN
         df_processed['H2O_ppm'] = (e_kPa / P_total_kPa) * 1e6
         df_processed.loc[P_total_kPa <= 0, 'H2O_ppm'] = np.nan # Handle zero/negative pressure explicitly

    print("H2O ppm calculation complete.")
    h2o_ppm_calculated = True
else:
    print("Warning: Cannot calculate H2O ppm. Missing one or more required columns: ", required_cols_for_ppm)
    h2o_ppm_calculated = False
    # Add a dummy column with NaNs if needed later, although outlier/aggregation will handle absence
    if 'H2O_ppm' not in df_processed.columns:
         df_processed['H2O_ppm'] = np.nan
# --- *** END ADDED SECTION *** ---


# Columns to process (updated to include H2O_ppm if calculated)
# We still keep SHT_Hum_% here if needed for other potential processing,
# but we won't plot it in the time series below. Add H2O_ppm.
sensor_columns = ['SHT_TempC', 'SHT_Hum_%', 'BMP_PressAtm', 'CO2_ppm']
if h2o_ppm_calculated and 'H2O_ppm' not in sensor_columns:
     sensor_columns.append('H2O_ppm') # Add the new column for processing

# 1. Outlier Removal (using IQR method per column)
print("Removing outliers...")
for col in sensor_columns:
    if col in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed[col]): # Check if column exists and is numeric
        Q1 = df_processed[col].quantile(0.25)
        Q3 = df_processed[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Replace outliers with NaN. NaNs will be ignored by mean/std in resampling.
        original_count = df_processed[col].count() # Count non-NaN before masking
        df_processed[col] = df_processed[col].mask(
            (df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)
        )
        nan_count = original_count - df_processed[col].count() # Count NaNs introduced
        # The original outlier calculation was slightly off, comparing df_plotting to df_processed bounds
        # This version calculates outliers identified within df_processed itself
        print(f"Identified and marked {nan_count} outliers as NaN in '{col}'.")
    elif col not in df_processed.columns:
         print(f"Skipping outlier removal for '{col}': Column not found.")
    else:
         print(f"Skipping outlier removal for '{col}': Column is not numeric.")


# 2. Averaging for 1 minute and calculating standard deviation
print("Aggregating data to 1-minute intervals (mean and std)...")
# The '1T' or '1min' string means 1-minute frequency for resampling.
# .agg will compute both mean and std. NaN values are skipped by default.
# Ensure we only try to aggregate columns that actually exist in df_processed
columns_to_aggregate = [col for col in sensor_columns if col in df_processed.columns]
if columns_to_aggregate:
    df_aggregated = df_processed[columns_to_aggregate].resample('2min').agg(['mean', 'std'])
else:
    print("Error: No columns available for aggregation.")
    # Handle error appropriately, maybe exit or create an empty df_aggregated
    df_aggregated = pd.DataFrame() # Create empty df to avoid downstream errors

# df_aggregated will now have multi-level columns, e.g., ('SHT_TempC', 'mean') and ('SHT_TempC', 'std')

# --- 1) Time Series Subplots (Modified) ---
print("Generating Plot 1: Time Series Data (2-min average with std bands)...")
fig1, axes1 = plt.subplots(4, 1, figsize=(15, 12), sharex=True)
fig1.suptitle('Sensor Time Series Data (2-min Average ±1 Std Dev)', fontsize=16, y=0.99)

# --- *** MODIFIED: Plot Configuration *** ---
# Replace SHT_Hum_% plot config with H2O_ppm config
plot_configs = [
    {'col': 'SHT_TempC', 'label': 'SHT Temp', 'color': 'crimson', 'ylabel': 'Temperature (°C)', 'title': 'Ambient Temperature (SHT85)'},
    {'col': 'H2O_ppm', 'label': 'Water Vapor', 'color': 'deepskyblue', 'ylabel': 'H2O (ppm)', 'title': 'Water Vapor Concentration'}, # MODIFIED ROW
    {'col': 'BMP_PressAtm', 'label': 'BMP Pressure', 'color': 'green', 'ylabel': 'Pressure (atm)', 'title': 'Atmospheric Pressure (BMP585)'},
    {'col': 'CO2_ppm', 'label': 'CO2', 'color': 'purple', 'ylabel': 'CO2 (ppm)', 'title': 'CO2 Concentration (SCD30)'}
]
# --- *** END MODIFIED SECTION *** ---


for i, config in enumerate(plot_configs):
    ax = axes1[i]
    col_name = config['col']

    # Check if the specific column (mean and std) exists in the aggregated data
    mean_col = (col_name, 'mean')
    std_col = (col_name, 'std')

    if mean_col not in df_aggregated.columns or std_col not in df_aggregated.columns:
        # Provide more specific feedback if the column wasn't calculated or aggregated
        if col_name == 'H2O_ppm' and not h2o_ppm_calculated:
             print(f"Warning: Cannot plot {col_name}. Data was not calculated (missing inputs).")
             ax.set_title(f"{config['title']} (Input Data Missing)")
        elif col_name not in columns_to_aggregate:
             print(f"Warning: Cannot plot {col_name}. Column was not found in processed data before aggregation.")
             ax.set_title(f"{config['title']} (Column Not Found)")
        else:
             print(f"Warning: Mean or std for {col_name} not found in aggregated data. Skipping plot.")
             ax.set_title(f"{config['title']} (Aggregation Error?)")

        ax.set_ylabel(config['ylabel'])
        if i == len(plot_configs) - 1: ax.set_xlabel('Time')
        ax.grid(True, linestyle='--', alpha=0.6) # Add grid for readability
        continue

    # Extract mean and std series
    mean_series = df_aggregated[mean_col].dropna() # Drop rows where mean is NaN
    std_series = df_aggregated[std_col].reindex(mean_series.index).fillna(0) # Align and fill NaN std with 0

    if mean_series.empty:
        print(f"Warning: No data to plot for {col_name} after aggregation and NaN removal.")
        ax.set_title(f"{config['title']} (No data after aggregation)")
        ax.set_ylabel(config['ylabel'])
        if i == len(plot_configs) - 1: ax.set_xlabel('Time')
        ax.grid(True, linestyle='--', alpha=0.6)
        continue

    # Plot the mean
    ax.plot(mean_series.index, mean_series, label=config['label'], color=config['color'], linewidth=1.5)

    # Plot the standard deviation bands
    ax.fill_between(mean_series.index,
                    mean_series - std_series,
                    mean_series + std_series,
                    color=config['color'],
                    alpha=0.2, # Semi-transparent
                    label='±1 Std Dev' if i == 0 else "_nolegend_") # Label std dev band only once

    ax.set_ylabel(config['ylabel'])
    ax.legend(loc='upper right')
    ax.set_title(config['title'])
    ax.grid(True, linestyle='--', alpha=0.6) # Add grid for readability

# Set common x-label only for the last plot
axes1[-1].set_xlabel('Time')

# Apply date formatting to the x-axis
if not df_aggregated.empty and pd.api.types.is_datetime64_any_dtype(df_aggregated.index):
    fig1.autofmt_xdate()
    # Use AutoDateLocator and ConciseDateFormatter for better automatic formatting
    locator = mdates.AutoDateLocator(minticks=5, maxticks=10)
    formatter = mdates.ConciseDateFormatter(locator)
    axes1[-1].xaxis.set_major_locator(locator)
    axes1[-1].xaxis.set_major_formatter(formatter)
    # axes1[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M')) # Alternative explicit format

plt.tight_layout(rect=[0, 0, 1, 0.97]) # Adjust rect to prevent title overlap

# Ensure output folder exists before saving
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

plot1_path = os.path.join(output_folder, "plot1_time_series_aggregated.png")
plt.savefig(plot1_path)
print(f"Saved: {plot1_path}")
plt.show()
# plt.close(fig1)

In [ ]:
# --- Data Preprocessing ---
print("Preprocessing data: calculating H2O ppm, removing outliers, and preparing for aggregation...")
df_processed = df_plotting.copy()

# Ensure Plot_Time is datetime and set as index
if not pd.api.types.is_datetime64_any_dtype(df_processed['Plot_Time']):
    df_processed['Plot_Time'] = pd.to_datetime(df_processed['Plot_Time'])
df_processed = df_processed.set_index('Plot_Time')

# --- *** ADDED: Calculate Water Vapor Concentration (ppm) *** ---
# Constants
ATM_TO_KPA = 101.325 # Conversion factor from atm to kPa

# Ensure required columns exist
required_cols_for_ppm = ['SHT_TempC', 'SHT_Hum_%', 'BMP_PressAtm']
if all(col in df_processed.columns for col in required_cols_for_ppm):
    print("Calculating Water Vapor Concentration (H2O ppm)...")
    # Extract data (use .values for slight speedup with numpy, though pandas ops are fine too)
    T_C = df_processed['SHT_TempC']
    RH_percent = df_processed['SHT_Hum_%']
    P_atm = df_processed['BMP_PressAtm']

    # 1. Calculate Saturation Vapor Pressure (es) in kPa using Magnus-Tetens approximation
    # es(T) = 0.6108 * exp((17.27 * T) / (T + 237.3))  (T in Celsius, es in kPa)
    es_kPa = 0.6108 * np.exp((17.27 * T_C) / (T_C + 237.3))

    # 2. Calculate Actual Vapor Pressure (e) in kPa
    # e = (RH / 100) * es
    e_kPa = (RH_percent / 100.0) * es_kPa

    # 3. Convert Total Atmospheric Pressure (P) to kPa
    P_total_kPa = P_atm * ATM_TO_KPA

    # 4. Calculate Water Vapor Concentration in ppm (volume/volume)
    # ppm_v = (e / P_total) * 1,000,000
    # Ensure P_total_kPa is not zero to avoid division errors (though unlikely for atm pressure)
    # Replace potential division by zero or NaN pressure with NaN result
    with np.errstate(divide='ignore', invalid='ignore'): # Suppress warnings for division by zero/NaN
         df_processed['H2O_ppm'] = (e_kPa / P_total_kPa) * 1e6
         df_processed.loc[P_total_kPa <= 0, 'H2O_ppm'] = np.nan # Handle zero/negative pressure explicitly

    print("H2O ppm calculation complete.")
    h2o_ppm_calculated = True
else:
    print("Warning: Cannot calculate H2O ppm. Missing one or more required columns: ", required_cols_for_ppm)
    h2o_ppm_calculated = False
    # Add a dummy column with NaNs if needed later, although outlier/aggregation will handle absence
    if 'H2O_ppm' not in df_processed.columns:
         df_processed['H2O_ppm'] = np.nan
# --- *** END ADDED SECTION *** ---


# Columns to process (updated to include H2O_ppm if calculated)
# We still keep SHT_Hum_% here if needed for other potential processing,
# but we won't plot it in the time series below. Add H2O_ppm.
# --- *** MODIFIED: Added BMP_TempC and CPU_TempC for aggregation *** ---
sensor_columns = ['SHT_TempC', 'SHT_Hum_%', 'BMP_PressAtm', 'CO2_ppm', 'BMP_TempC', 'CPU_TempC']
# --- *** END MODIFIED SECTION *** ---
if h2o_ppm_calculated and 'H2O_ppm' not in sensor_columns:
     sensor_columns.append('H2O_ppm') # Add the new column for processing

# 1. Outlier Removal (using IQR method per column)
print("Removing outliers...")
for col in sensor_columns:
    if col in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed[col]): # Check if column exists and is numeric
        Q1 = df_processed[col].quantile(0.25)
        Q3 = df_processed[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Replace outliers with NaN. NaNs will be ignored by mean/std in resampling.
        original_count = df_processed[col].count() # Count non-NaN before masking
        df_processed[col] = df_processed[col].mask(
            (df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)
        )
        nan_count = original_count - df_processed[col].count() # Count NaNs introduced
        # The original outlier calculation was slightly off, comparing df_plotting to df_processed bounds
        # This version calculates outliers identified within df_processed itself
        print(f"Identified and marked {nan_count} outliers as NaN in '{col}'.")
    elif col not in df_processed.columns:
         print(f"Skipping outlier removal for '{col}': Column not found.")
    else:
         print(f"Skipping outlier removal for '{col}': Column is not numeric.")


# 2. Averaging for 1 minute and calculating standard deviation
print("Aggregating data to 1-minute intervals (mean and std)...")
# The '1T' or '1min' string means 1-minute frequency for resampling.
# .agg will compute both mean and std. NaN values are skipped by default.
# Ensure we only try to aggregate columns that actually exist in df_processed
columns_to_aggregate = [col for col in sensor_columns if col in df_processed.columns]
if columns_to_aggregate:
    df_aggregated = df_processed[columns_to_aggregate].resample('2min').agg(['mean', 'std'])
else:
    print("Error: No columns available for aggregation.")
    # Handle error appropriately, maybe exit or create an empty df_aggregated
    df_aggregated = pd.DataFrame() # Create empty df to avoid downstream errors

# df_aggregated will now have multi-level columns, e.g., ('SHT_TempC', 'mean') and ('SHT_TempC', 'std')

# --- 1) Time Series Subplots (Modified) ---
print("Generating Plot 1: Time Series Data (2-min average with std bands)...")
fig1, axes1 = plt.subplots(4, 1, figsize=(15, 12), sharex=True)
fig1.suptitle('Sensor Time Series Data (2-min Average ±1 Std Dev)', fontsize=16, y=0.99)

# --- *** MODIFIED: Plot Configuration *** ---
# Replace SHT_Hum_% plot config with H2O_ppm config
# Note: BMP_TempC and CPU_TempC are added directly to the first plot, not via this config list
plot_configs = [
    {'col': 'SHT_TempC', 'label': 'SHT Temp', 'color': 'crimson', 'ylabel': 'Temperature (°C)', 'title': 'Ambient & Sensor Temperatures'}, # MODIFIED Title
    {'col': 'H2O_ppm', 'label': 'Water Vapor', 'color': 'deepskyblue', 'ylabel': 'H2O (ppm)', 'title': 'Water Vapor Concentration'}, # MODIFIED ROW
    {'col': 'BMP_PressAtm', 'label': 'BMP Pressure', 'color': 'green', 'ylabel': 'Pressure (atm)', 'title': 'Atmospheric Pressure (BMP585)'},
    {'col': 'CO2_ppm', 'label': 'CO2', 'color': 'purple', 'ylabel': 'CO2 (ppm)', 'title': 'CO2 Concentration (SCD30)'}
]
# --- *** END MODIFIED SECTION *** ---


for i, config in enumerate(plot_configs):
    ax = axes1[i]
    col_name = config['col']

    # Check if the specific column (mean and std) exists in the aggregated data
    mean_col = (col_name, 'mean')
    std_col = (col_name, 'std')

    if mean_col not in df_aggregated.columns or std_col not in df_aggregated.columns:
        # Provide more specific feedback if the column wasn't calculated or aggregated
        if col_name == 'H2O_ppm' and not h2o_ppm_calculated:
             print(f"Warning: Cannot plot {col_name}. Data was not calculated (missing inputs).")
             ax.set_title(f"{config['title']} (Input Data Missing)")
        elif col_name not in columns_to_aggregate:
             print(f"Warning: Cannot plot {col_name}. Column was not found in processed data before aggregation.")
             ax.set_title(f"{config['title']} (Column Not Found)")
        else:
             print(f"Warning: Mean or std for {col_name} not found in aggregated data. Skipping plot.")
             ax.set_title(f"{config['title']} (Aggregation Error?)")

        ax.set_ylabel(config['ylabel'])
        if i == len(plot_configs) - 1: ax.set_xlabel('Time')
        ax.grid(True, linestyle='--', alpha=0.6) # Add grid for readability
        continue

    # Extract mean and std series
    mean_series = df_aggregated[mean_col].dropna() # Drop rows where mean is NaN
    std_series = df_aggregated[std_col].reindex(mean_series.index).fillna(0) # Align and fill NaN std with 0

    if mean_series.empty:
        print(f"Warning: No data to plot for {col_name} after aggregation and NaN removal.")
        ax.set_title(f"{config['title']} (No data after aggregation)")
        ax.set_ylabel(config['ylabel'])
        if i == len(plot_configs) - 1: ax.set_xlabel('Time')
        ax.grid(True, linestyle='--', alpha=0.6)
        continue

    # Plot the mean for the primary variable of this subplot
    ax.plot(mean_series.index, mean_series, label=config['label'], color=config['color'], linewidth=1.5)

    # Plot the standard deviation bands ONLY for the primary variable
    ax.fill_between(mean_series.index,
                    mean_series - std_series,
                    mean_series + std_series,
                    color=config['color'],
                    alpha=0.2, # Semi-transparent
                    label='±1 Std Dev' if i == 0 else "_nolegend_") # Label std dev band only once for SHT_TempC

    # --- *** ADDED: Plot BMP_TempC and CPU_TempC on the first subplot (i=0) *** ---
    if i == 0:
        # Plot BMP_TempC mean if available
        bmp_temp_mean_col = ('BMP_TempC', 'mean')
        if bmp_temp_mean_col in df_aggregated.columns:
            bmp_temp_mean_series = df_aggregated[bmp_temp_mean_col].dropna()
            if not bmp_temp_mean_series.empty:
                ax.plot(bmp_temp_mean_series.index, bmp_temp_mean_series, label='BMP Temp', color='orange', linewidth=1.0, linestyle='--')
            else:
                print(f"Warning: No data to plot for BMP_TempC after aggregation and NaN removal.")
        else:
            print(f"Warning: BMP_TempC mean not found in aggregated data. Skipping its plot line.")

        # Plot CPU_TempC mean if available
        cpu_temp_mean_col = ('CPU_TempC', 'mean')
        if cpu_temp_mean_col in df_aggregated.columns:
            cpu_temp_mean_series = df_aggregated[cpu_temp_mean_col].dropna()
            if not cpu_temp_mean_series.empty:
                 ax.plot(cpu_temp_mean_series.index, cpu_temp_mean_series, label='CPU Temp', color='gray', linewidth=1.0, linestyle=':')
            else:
                print(f"Warning: No data to plot for CPU_TempC after aggregation and NaN removal.")
        else:
             print(f"Warning: CPU_TempC mean not found in aggregated data. Skipping its plot line.")
    # --- *** END ADDED SECTION *** ---

    ax.set_ylabel(config['ylabel'])
    ax.legend(loc='upper right')
    ax.set_title(config['title'])
    ax.grid(True, linestyle='--', alpha=0.6) # Add grid for readability

# Set common x-label only for the last plot
axes1[-1].set_xlabel('Time')

# Apply date formatting to the x-axis
if not df_aggregated.empty and pd.api.types.is_datetime64_any_dtype(df_aggregated.index):
    fig1.autofmt_xdate()
    # Use AutoDateLocator and ConciseDateFormatter for better automatic formatting
    locator = mdates.AutoDateLocator(minticks=5, maxticks=10)
    formatter = mdates.ConciseDateFormatter(locator)
    axes1[-1].xaxis.set_major_locator(locator)
    axes1[-1].xaxis.set_major_formatter(formatter)
    # axes1[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M')) # Alternative explicit format

plt.tight_layout(rect=[0, 0, 1, 0.97]) # Adjust rect to prevent title overlap

# Ensure output folder exists before saving
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

plot1_path = os.path.join(output_folder, "plot1_time_series_aggregated.png")
plt.savefig(plot1_path)
print(f"Saved: {plot1_path}")
plt.show()
# plt.close(fig1)

In [ ]:
%matplotlib widget

In [ ]:
# --- 3) 3D Coordinates Plot ---
print("Generating Plot 3: 3D GPS Coordinates...")
gps_fixed_data = df_plotting[df_plotting['GPS_Fix'] == 1].copy()

coord_cols = ['GPS_Lon', 'GPS_Lat', 'GPS_Alt_m']
for col in coord_cols:
    gps_fixed_data[col] = pd.to_numeric(gps_fixed_data[col], errors='coerce')

gps_fixed_data.dropna(subset=coord_cols, inplace=True)

gps_fixed_data = gps_fixed_data[~((gps_fixed_data['GPS_Lon'] == 0) & \
                                  (gps_fixed_data['GPS_Lat'] == 0) & \
                                  (gps_fixed_data['GPS_Alt_m'] == 0))]

if not gps_fixed_data.empty and len(gps_fixed_data) > 1:
    fig3 = plt.figure(figsize=(10, 8))
    ax3d = fig3.add_subplot(111, projection='3d')
    
    ax3d.plot(gps_fixed_data['GPS_Lon'], 
              gps_fixed_data['GPS_Lat'], 
              gps_fixed_data['GPS_Alt_m'], 
              label='GPS Track', marker='o', markersize=2, linestyle='-')
    
    ax3d.scatter(gps_fixed_data['GPS_Lon'].iloc[0], gps_fixed_data['GPS_Lat'].iloc[0], gps_fixed_data['GPS_Alt_m'].iloc[0],
                 color='green', s=50, label='Start', depthshade=True)
    ax3d.scatter(gps_fixed_data['GPS_Lon'].iloc[-1], gps_fixed_data['GPS_Lat'].iloc[-1], gps_fixed_data['GPS_Alt_m'].iloc[-1],
                 color='red', s=50, label='End', depthshade=True)

    ax3d.set_xlabel('Longitude (°)')
    ax3d.set_ylabel('Latitude (°)')
    ax3d.set_zlabel('Altitude (m)')
    ax3d.set_title('3D GPS Track (where Fix=1)', fontsize=16)
    ax3d.legend()
    
    x_range = gps_fixed_data['GPS_Lon'].max() - gps_fixed_data['GPS_Lon'].min()
    y_range = gps_fixed_data['GPS_Lat'].max() - gps_fixed_data['GPS_Lat'].min()
    z_range = gps_fixed_data['GPS_Alt_m'].max() - gps_fixed_data['GPS_Alt_m'].min()
    max_range = max(x_range, y_range, z_range if z_range > 0 else (x_range+y_range)/2 if (x_range+y_range)>0 else 1 ) 
    if max_range == 0: max_range = 1 

    mid_x = (gps_fixed_data['GPS_Lon'].max() + gps_fixed_data['GPS_Lon'].min()) * 0.5
    mid_y = (gps_fixed_data['GPS_Lat'].max() + gps_fixed_data['GPS_Lat'].min()) * 0.5
    mid_z = (gps_fixed_data['GPS_Alt_m'].max() + gps_fixed_data['GPS_Alt_m'].min()) * 0.5
    
    if x_range > 0 : ax3d.set_xlim(mid_x - max_range * 0.5, mid_x + max_range * 0.5)
    if y_range > 0 : ax3d.set_ylim(mid_y - max_range * 0.5, mid_y + max_range * 0.5)
    if z_range > 0 : ax3d.set_zlim(mid_z - max_range * 0.5, mid_z + max_range * 0.5)
    elif 'GPS_Alt_m' in gps_fixed_data and not gps_fixed_data['GPS_Alt_m'].empty:
        alt_min = gps_fixed_data['GPS_Alt_m'].min()
        alt_max = gps_fixed_data['GPS_Alt_m'].max()
        if alt_min == alt_max:
             ax3d.set_zlim(alt_min - 1, alt_max + 1) # Provide a small range if all altitudes are same
        else: # Should be covered by z_range > 0 but as a fallback
             ax3d.set_zlim(alt_min, alt_max)


    plt.tight_layout()
    plot3_path = os.path.join(output_folder, "plot3_3d_coordinates.png")
    plt.savefig(plot3_path)
    print(f"Saved: {plot3_path}")
    plt.show() # MODIFIED: Show plot
    # plt.close(fig3) # MODIFIED: Commented out
elif gps_fixed_data.empty:
    print("No data with GPS_Fix=1 and valid coordinates found. Skipping 3D coordinate plot.")
else: # Only one point with GPS fix
    print("Only one data point with GPS_Fix=1 and valid coordinates found. Skipping 3D line plot, plotting single point.")
    fig3 = plt.figure(figsize=(10, 8))
    ax3d = fig3.add_subplot(111, projection='3d')
    ax3d.scatter(gps_fixed_data['GPS_Lon'].iloc[0], gps_fixed_data['GPS_Lat'].iloc[0], gps_fixed_data['GPS_Alt_m'].iloc[0],
                 color='blue', s=50, label='Single GPS Point', depthshade=True)
    ax3d.set_xlabel('Longitude (°)')
    ax3d.set_ylabel('Latitude (°)')
    ax3d.set_zlabel('Altitude (m)')
    ax3d.set_title('3D GPS Point (where Fix=1)', fontsize=16)
    ax3d.legend()
    plt.tight_layout()
    plot3_path = os.path.join(output_folder, "plot3_3d_coordinates.png")
    plt.savefig(plot3_path)
    print(f"Saved: {plot3_path}")
    plt.show() # MODIFIED: Show plot
    # plt.close(fig3) # MODIFIED: Commented out

print("\nFinished generating and saving plots.")